In [1]:
# REQUIRED ARGUMENTS
RUN_NAME = 'test1'
CONTROL_TIMEPOINT_CSV = 'timepoint_csv/timepoints.csv'
CONTROL_FASTQ_DIR = 'fastqs'
CONTROL_FASTQ = 'SM-1_S73'
PAM_ORIENTATION = 'three_prime' # or 'five_prime'
PAM_LENGTH = 4
PAM_START = 0

def reverse_complement(seq):
    nt_complement = dict({'A':'T','C':'G','G':'C','T':'A','N':'N','_':'_','-':'-'})
    return "".join([nt_complement[c] for c in seq.upper()[-1::-1]])
    
# ADDITIONAL ARGUMENTS (with default values entered)
MAX_PAM_LENGTH = 4
SPACERS = {'oLS-1874':'AGTCTCCTCAGCAAAACGAA'}
P5_TIMEPOINT_BARCODE_START = 0
P7_TIMEPOINT_BARCODE_START = 0

import pandas as pd

timepoint_barcodes = pd.DataFrame({'timepoint': [0], 'description': ['qc-control'], 'P5_timepoint_barcode': ['GTCAGT'], 'P7_timepoint_barcode': ['GTCAGT']})
timepoint_barcodes.set_index('timepoint', inplace=True)
timepoint_barcodes.to_csv(CONTROL_TIMEPOINT_CSV)

"""
Argument definitions:

`RUN_NAME`: Provide a name for the run (no spaces or special characters).

`BARCODE_CSV`: Filepath to a CSV with the control barcode information and the following headers:

|sample	            |description|P5_sample_barcode|P7_sample_barcode|
|:------------------|:----------|:---------------:|:---------------:|
|unique_sample_id_01|control    |ATGC             |TCGC             |

Barcodes should be provided 5' to 3' as they appear in the primer sequences that encode them. Avoid the use of special characters.

`FASTQ_DIR`: Folder directory contianing fastq files.

`CONTROL_FASTQ`: Abbreviated name of the fastq files for the library. Fastq names must be abbreviated as follows:

|fastq complete file names       |abbreviated name|
|:-------------------------------|:--------------:|
|control_S1_L001_R1_001.fastq.gz |timepoint1_S1   |
|control_S1_L001_R2_001.fastq.gz |                |
|control_S1_L002_R1_001.fastq.gz |                |
|control_S1_L002_R2_001.fastq.gz |                |
|control_S1_L003_R1_001.fastq.gz |                |
|control_S1_L003_R2_001.fastq.gz |                |
|control_S1_L004_R1_001.fastq.gz |                |
|control_S1_L004_R2_001.fastq.gz |                |

`PAM_ORIENTATION`: Orientation of the PAM relative to the spacer (`three_prime` (Cas9) or `five_prime` (Cas12)). 

`PAM_LENGTH`: Number of bases to include in the PAM.

`PAM_START`: Position relative to the spacer to begin defining the PAM. Enter `0` to start immediately adjacent to the spacer. 
             Many CRISPR-Cas nucleases have extended PAM preferences that may make it advantageous to consider more distal 
             positions as the "start" of the PAM. 
             For example, for the canonical *Staphylococcus aureus* Cas9 PAM of "NNGRRT", 
             it may be desirable to define `PAM_START` = 2, to consider a 4 nucleotide "GRRT" PAM, rather than the full 6 nucleotides.

Examples for specifying the PAM sequence:

Specifying the `PAM_LENGTH` and `PAM_START` arguments for an example spacer and `three_prime` PAM orientation: 

    5'-[SPACER][PAM]-3'    
    5'-[SPACER]CGGTA-3'
               01234  (PAM_START)
Examples:

|PAM_LENGTH|PAM_START|PAM sequence|
|:--------:|:-------:|:----------:|
|3         |0        |CGG         |
|4         |0        |CGGT        |
|3         |1        |GGT         |

Specifying the `PAM_LENGTH` and `PAM_START` arguments for an example spacer and `five_prime` PAM orientation: 

    5'-[PAM][SPACER]-3'    
    5'-CTTTA[SPACER]-3'
       43210 (PAM_START)
Examples:

|PAM_LENGTH|PAM_START|PAM sequence|
|:--------:|:-------:|:----------:|
|5         |0        |CTTTA       |
|4         |0        |TTTA        |
|4         |1        |CTTT        |


`CONTROL_SAMPLE`: Unique sample ID of the untreated randomized PAM library control sample.

`MAX_PAM_LENGTH`: Integer. The maximum PAM length that will be considered, starting immediately adjacent to the spacer. PAMs of this length will be enumerated from fastq files. PAMs will later be truncated to the bases indicated by `PAM_START` and `PAM_LENGTH`. Default value is `8`. 

`SPACERS`: Dictionary with key = spacer name, value = spacer sequence. Default spacers: `{'SPACER1':'GGGCACGGGCAGCTTGCCGG', 'SPACER2':'GTCGCCCTCGAACTTCACCT'}`
           
`P5_SAMPLE_BARCODE_START`: Integer, location of the 5' end of the P5 sample barcode. Default = `2`.
        
`P7_SAMPLE_BARCODE_START`: Integer, location of the 5' end of the P7 sample barcode. Default = `2`.

Example P5 and P7 barcodes in an example amplicon for a 3' PAM library below. Barcodes are the capitalized "NNNN" sequences, read from 5' to 3' on their respective strands:

                 P7 BARCODE                                                
    POSITION:  012
            5' --NNNN-----------------[SPACER][PAM]-----------------nnnn-- 3'
            3' --nnnn-----------------[SPACER][PAM]-----------------NNNN-- 5'
                                                        POSITION:      210
                                                                    P5 BARCODE
"""
pass

In [2]:
timepoint_barcodes

,description,P5_timepoint_barcode,P7_timepoint_barcode
timepoint,,,
0,qc-control,GTCAGT,GTCAGT


In [10]:
import os
import sys
sys.path.append('../code')
%load_ext autoreload
%autoreload 2
import PAMDA

import cProfile

pr = cProfile.Profile()
pr.enable()

def runqc():
    PAMDA.library_QC(RUN_NAME, 
                CONTROL_FASTQ_DIR,
                CONTROL_FASTQ,
                'test',
                PAM_ORIENTATION,
                PAM_LENGTH,
                PAM_START,
                MAX_PAM_LENGTH, 
                SPACERS,
                P5_TIMEPOINT_BARCODE_START,
                P7_TIMEPOINT_BARCODE_START)
    
runqc()

pr.disable()
pr.print_stats(sort='time')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Begin library QC
/home/alex/eth/spcas9/pamda/tests
['fastqs/test/SM-1_S73_L001_R1_001.fastq.gz']
Not using timepoints


  0%|          | 0/1 [00:00<?, ?it/s]

79.61% of reads mapped from SM-1_S73 (22326 reads)
Total Reads: 28043, Wrong Spacer: 5717, Wrong Barcode: 0, Unknown Timeppoint: 0
writing compressed CSV output
summarizing raw read counts
grouping counts by indicated PAM bases
PAM library oLS-1874 max:min ratio: 7.1429
PAM library oLS-1874 90:10 ratio: 2.5882
PAM library oLS-1874 skewness: 0.969
Library QC complete
         322242 function calls (314008 primitive calls) in 5.481 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       70    4.498    0.064    4.498    0.064 {method 'acquire' of '_thread.lock' objects}
      224    0.052    0.000    0.052    0.000 {method 'set_text' of 'matplotlib.ft2font.FT2Font' objects}
        4    0.043    0.011    0.043    0.011 {built-in method posix.fork}
       40    0.023    0.001    0.023    0.001 socket.py:621(send)
     2302    0.017    0.000    0.017    0.000 {method 'reduce' of 'numpy.ufunc' objects}
3438/3038    0.014    0.000 

In [11]:
TIMEPOINT_CSV = 'timepoint_csv/timepoints.csv'

USE_TIMEPOINTS = None
TOP_N_NORMALIZE = 5
INIT_RATE_EST = [0.0001, 0.001, 0.01]
READ_SUM_MIN = 4
TPS_SUM_MIN = 1
PAM1_NT_RANK = {1:'A',2:'C',3:'G',4:'T'}
PAM2_NT_RANK = {1:'A',2:'C',3:'G',4:'T'}
PAM1_INDEX_RANK = None
PAM2_INDEX_RANK = None
AVERAGE_SPACER = False
HEATMAP_FIXED_MIN = -3
HEATMAP_FIXED_MAX = -1
LOG_SCALE_HEATMAP = True
CSV_INPUT_RAW_COUNT = None
CSV_INPUT_NORM_COUNT = None
CSV_INPUT_RATES = None

raw_count_csv = f'output/test1/PAMDA_1_raw_counts.csv'
PAMDA.PAMDA_complete('test',
             TIMEPOINT_CSV,
             CONTROL_FASTQ_DIR,
             {CONTROL_FASTQ: 'test'},
             PAM_ORIENTATION,
             PAM_LENGTH,
             PAM_START,
             raw_count_csv, # randomized PAM library control
             'test', # control sample
             TIMEPOINTS = [0],
             MAX_PAM_LENGTH = MAX_PAM_LENGTH,
             SPACERS = SPACERS,
             P5_TIMEPOINT_BARCODE_START = P5_TIMEPOINT_BARCODE_START,
             P7_TIMEPOINT_BARCODE_START = P7_TIMEPOINT_BARCODE_START,
             USE_TIMEPOINTS = USE_TIMEPOINTS,
             TOP_N_NORMALIZE = TOP_N_NORMALIZE,
             INIT_RATE_EST = INIT_RATE_EST,
             READ_SUM_MIN = READ_SUM_MIN,
             TPS_SUM_MIN = TPS_SUM_MIN,
             PAM1_NT_RANK = PAM1_NT_RANK,
             PAM2_NT_RANK = PAM2_NT_RANK,
             PAM1_INDEX_RANK = PAM1_INDEX_RANK,
             PAM2_INDEX_RANK = PAM2_INDEX_RANK,
             AVERAGE_SPACER = AVERAGE_SPACER,
             HEATMAP_FIXED_MIN = HEATMAP_FIXED_MIN,
             HEATMAP_FIXED_MAX = HEATMAP_FIXED_MAX,
             LOG_SCALE_HEATMAP = LOG_SCALE_HEATMAP)

BEGIN: generate counts from fastqs
{'GTCAGT': 0}


  0%|          | 0/1 [00:00<?, ?it/s]

0.0% of reads mapped from SM-1_S73 (0 reads)
Total Reads: 28043, Wrong Spacer: 5717, Wrong Barcode: 621, Unknown Timeppoint: 1
writing compressed CSV output
summarizing raw read counts
FINISHED: generate counts from fastqs 

BEGIN: convert raw counts to normalized counts
['test']
grouping counts by indicated PAM bases


sample::   0%|          | 0/1 [00:00<?, ?it/s]

Control Sample test
['test']


KeyError: 'Norm_Counts_1'

In [ ]:
'GTCAGTAGGTACCCTCGTGACCTGCGCAGAGCATTCGTTTTGCTGAGGAGACTCCTCCGGTACCGGATCCAGTCGACTGAATTGGTTCCTTTAAAGCCTGCTTTTTTGTACAGCTAGCGACGACTGA'.find('TCAGTCGTCGCT')

-1

In [ ]:
[] is []

False